### Cleaning & Preprocessing

In [1]:
import pandas as pd 
import re
import numpy as np 
import pickle 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("goibibo_flights_data_predictions.csv")
df.head()

,flight date,airline,flight_num,class,from,dep_time,to,arr_time,duration,price,stops,Unnamed: 11,Unnamed: 12
0,26-06-2023,SpiceJet,SG-8709,economy,Delhi,18:55,Mumbai,21:05,02h 10m,"6,013",non-stop,NaN,NaN
1,26-06-2023,SpiceJet,SG-8157,economy,Delhi,06:20,Mumbai,08:40,02h 20m,"6,013",non-stop,NaN,NaN
2,26-06-2023,AirAsia,I5-764,economy,Delhi,04:25,Mumbai,06:35,02h 10m,"6,016",non-stop,NaN,NaN
3,26-06-2023,Vistara,UK-995,economy,Delhi,10:20,Mumbai,12:35,02h 15m,"6,015",non-stop,NaN,NaN
4,26-06-2023,Vistara,UK-963,economy,Delhi,08:50,Mumbai,11:10,02h 20m,"6,015",non-stop,NaN,NaN


In [4]:
df.columns 

Index(['flight date', 'airline', 'flight_num', 'class', 'from', 'dep_time',
       'to', 'arr_time', 'duration', 'price', 'stops', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')

In [5]:
def drop_column(df, column_names):
    """
    Drop specified columns from a dataframe if they exist.

    Args:
        df (DataFrame): The input dataframe.
        column_names (list): A list of column names to drop.

    Returns:
        DataFrame: The dataframe after dropping specified columns.
    """
    for column_name in column_names:
        if column_name in df.columns:
            df = df.drop(columns=[column_name])
            print(f"Column: {column_name} dropped successfully.")
        else:
            print(f"Column: {column_name} not found.")
    return df

# Call the function
df = drop_column(df, ['Unnamed: 11', 'Unnamed: 12'])
df


Column: Unnamed: 11 dropped successfully.
Column: Unnamed: 12 dropped successfully.


,flight date,airline,flight_num,class,from,dep_time,to,arr_time,duration,price,stops
0,26-06-2023,SpiceJet,SG-8709,economy,Delhi,18:55,Mumbai,21:05,02h 10m,"6,013",non-stop
1,26-06-2023,SpiceJet,SG-8157,economy,Delhi,06:20,Mumbai,08:40,02h 20m,"6,013",non-stop
2,26-06-2023,AirAsia,I5-764,economy,Delhi,04:25,Mumbai,06:35,02h 10m,"6,016",non-stop
3,26-06-2023,Vistara,UK-995,economy,Delhi,10:20,Mumbai,12:35,02h 15m,"6,015",non-stop
4,26-06-2023,Vistara,UK-963,economy,Delhi,08:50,Mumbai,11:10,02h 20m,"6,015",non-stop
...,...,...,...,...,...,...,...,...,...,...,...
300256,13-08-2023,Vistara,UK-822,business,Chennai,09:45,Hyderabad,19:50,10h 05m,"70,304",1-stop
300257,13-08-2023,Vistara,UK-826,business,Chennai,12:30,Hyderabad,22:55,10h 25m,"78,262",1-stop
300258,13-08-2023,Vistara,UK-832,business,Chennai,07:05,Hyderabad,20:55,13h 50m,"80,285",1-stop
300259,13-08-2023,Vistara,UK-828,business,Chennai,07:00,Hyderabad,17:00,10h 00m,"82,809",1-stop


In [6]:
df.columns

Index(['flight date', 'airline', 'flight_num', 'class', 'from', 'dep_time',
       'to', 'arr_time', 'duration', 'price', 'stops'],
      dtype='object')

In [7]:
df.isnull().sum()

flight date    0
airline        0
flight_num     0
class          0
from           0
dep_time       0
to             0
arr_time       0
duration       0
price          0
stops          0
dtype: int64

In [8]:
# Step 2: Convert 'flight date' to datetime format and extract features
df.rename(columns={"flight date": "Flight_Date"}, inplace=True)
df["Flight_Date"] = pd.to_datetime(df["Flight_Date"], format="%d-%m-%Y")

In [9]:
# Extract date, month, and year
df["Date"] = df["Flight_Date"].dt.day
df["Month"] = df["Flight_Date"].dt.month
df["Year"] = df["Flight_Date"].dt.year

In [10]:
# Step 3: Convert price column to numeric format if necessary
if df["price"].dtype == "object":
    df["price"] = df["price"].str.replace(",", "").astype(int)

In [11]:
# Step 4: Clean and convert 'duration' to total minutes
def clean_duration(duration):
    match = re.search(r"(\d{1,2})\.?\d{0,2}?h?\s?(\d{1,2})?m?", duration)
    if match:
        h = int(match.group(1)) if match.group(1) else 0
        m = int(match.group(2)) if match.group(2) else 0
        return h * 60 + m
    return None  # Return None if format is invalid

if "duration" in df.columns:
    df["duration_minutes"] = df["duration"].apply(clean_duration)
    df.drop(columns=["duration"], inplace=True)

In [12]:
# Step 5: Convert 'stops' to numerical format
import pandas as pd
import re

# Function to clean the 'stops' column
def clean_stops(value):
    value = str(value).strip()  # Remove leading/trailing spaces
    value = re.sub(r"\s*Via.*", "", value)  # Remove 'Via' and extra text
    return value

# Apply cleaning function
df["stops"] = df["stops"].apply(clean_stops)

# Convert 'stops' to numerical format
df["stops"] = df["stops"].replace({"non-stop": 0, "1-stop": 1, "2+-stop": 2}).fillna(1).astype(int)



C:\Users\chava\AppData\Local\Temp\ipykernel_3520\634987552.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["stops"] = df["stops"].replace({"non-stop": 0, "1-stop": 1, "2+-stop": 2}).fillna(1).astype(int)


In [13]:
# Dictionary mapping old column names to new ones
column_mapping = {
    'flight date': 'Flight_Date',
    'airline': 'Airline_Name',
    'flight_num': 'Flight_Number',
    'class': 'Travel_Class',
    'from': 'Origin_City',
    'dep_time': 'Departure_Time',
    'to': 'Destination_City',
    'arr_time': 'Arrival_Time',
    'duration': 'Flight_Duration',
    'price': 'Ticket_Price',
    'stops': 'Num_Stops',
    'flight date_day': 'Date_Day',
    'flight date_month': 'Date_Month',
    'flight date_quarter': 'Date_Quarter',
    'flight date_year': 'Date_Year',
    'departure_hour': 'Departure_Hour'
}

# Renaming columns
df = df.rename(columns=column_mapping)


In [14]:
df.columns

Index(['Flight_Date', 'Airline_Name', 'Flight_Number', 'Travel_Class',
       'Origin_City', 'Departure_Time', 'Destination_City', 'Arrival_Time',
       'Ticket_Price', 'Num_Stops', 'Date', 'Month', 'Year',
       'duration_minutes'],
      dtype='object')

In [15]:
df.dtypes

Flight_Date         datetime64[ns]
Airline_Name                object
Flight_Number               object
Travel_Class                object
Origin_City                 object
Departure_Time              object
Destination_City            object
Arrival_Time                object
Ticket_Price                 int64
Num_Stops                    int64
Date                         int32
Month                        int32
Year                         int32
duration_minutes             int64
dtype: object

In [16]:
# Ensure 'flight_date' is in datetime format
df['Flight_Date'] = pd.to_datetime(df['Flight_Date'], errors='coerce')

# Create 'departure_hour' for analysis
df['Departure_Hour'] = pd.to_datetime(df['Departure_Time'], format='%H:%M', errors='coerce').dt.hour

In [17]:
df.dtypes

Flight_Date         datetime64[ns]
Airline_Name                object
Flight_Number               object
Travel_Class                object
Origin_City                 object
Departure_Time              object
Destination_City            object
Arrival_Time                object
Ticket_Price                 int64
Num_Stops                    int64
Date                         int32
Month                        int32
Year                         int32
duration_minutes             int64
Departure_Hour               int32
dtype: object

In [18]:
def save_clean_csv(df,file_name):
    """
    Save the cleaned DataFrame to a CSV file.

    Args:
        df (DataFrame): The cleaned DataFrame to save.
        file_name (str): The name of the CSV file to save.

    Returns:
        None
    """
    try:
         #Save Dataframe to a CSV file
         df.to_csv(file_name, index=False)
         print(f"Cleaned DataFrame successfully saved as: {"Flight_Price_Cleaned_Preprocessing"} ")
    except Exception as e:
        print(f"An error occurred while saving the file: {'e'}")

save_clean_csv(df, "Flight_Price_cleaned_data_3.csv")

Cleaned DataFrame successfully saved as: Flight_Price_Cleaned_Preprocessing 


In [3]:
df2 = pd.read_csv(r"C:\Users\chava\OneDrive\Desktop\Behavioral_Data_Analyst\Flight_Price_cleaned_data_3.csv")
df2.columns


Index(['Flight_Date', 'Airline_Name', 'Flight_Number', 'Travel_Class',
       'Origin_City', 'Departure_Time', 'Destination_City', 'Arrival_Time',
       'Ticket_Price', 'Num_Stops', 'Date', 'Month', 'Year',
       'duration_minutes', 'Departure_Hour'],
      dtype='object')

In [20]:
df2['duration_minutes']=df2.rename(columns={'duration_minutes': 'Duration_Minutes'}, inplace=True)

In [7]:
df2['Destination_City'].unique()

array(['Mumbai', 'Bangalore', 'Kolkata', 'Hyderabad', 'Chennai', 'Delhi'],
      dtype=object)

In [8]:
df2.columns


Index(['Flight_Date', 'Airline_Name', 'Flight_Number', 'Travel_Class',
       'Origin_City', 'Departure_Time', 'Destination_City', 'Arrival_Time',
       'Ticket_Price', 'Num_Stops', 'Date', 'Month', 'Year',
       'duration_minutes', 'Departure_Hour'],
      dtype='object')

In [10]:
# Select features (independent variables)
Features = ["Airline_Name", "Origin_City", "Destination_City", "Num_Stops", "Travel_Class","Date", "Month","Year", "duration_minutes"]
Target = "Ticket_Price"  # Dependent variable

In [11]:
with open("Features_Data_2.pkl", "wb") as file:
    pickle.dump(Features, file)

print("File saved successfully!")

File saved successfully!


In [24]:
# One-Hot Encoding for categorical variables
df_encoding = pd.get_dummies(df2[Features], drop_first=True)

In [25]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(df_encoding, df2[Target], test_size=0.2, random_state=42)
